In [168]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import TensorBoard

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import os
import sys

In [169]:
import tensorflow as tf
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import cv2, os, glob

np.set_printoptions(threshold=np.inf) # NumPy 설정
os.environ["CUDA_VISIBLE_DEVICES"] = "3" # CUDA 가시성 설정 (GPU 사용시 필요)

# GPU 메모리 구성 설정 (GPU 사용시 필요)
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # TensorFlow 2.x에서는 set_memory_growth 대신에 메모리 할당 옵션을 설정하는 코드
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e)

In [170]:
# Custom abTanh Layer
class abTanh(layers.Layer):
    def __init__(self, units=None, init_a=7, init_b=0, **kwargs):
        super(abTanh, self).__init__(**kwargs)
        self.units = units
        self.init_a = init_a
        self.init_b = init_b

    def build(self, input_shape):
        units_shape = self.units if self.units else input_shape[1:]
        self.b = self.add_weight(shape=units_shape, initializer=tf.constant_initializer(self.init_b), trainable=True, name="b")
        self.a = self.add_weight(shape=units_shape, initializer=tf.constant_initializer(self.init_a), trainable=True, name="a")

    def call(self, inputs, **kwargs):
        x = tf.subtract(inputs, self.b)
        x = tf.nn.tanh(x)
        x = tf.multiply(x, self.a)
        return x

In [171]:
# Data preparation
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]], dtype=np.float32)
y = np.array([[0], [1], [1], [0]], dtype=np.float32)

In [172]:
# Model definition
model = models.Sequential()
model.add(layers.Input(shape=(2,)))
model.add(layers.Dense(10))
model.add(abTanh(units=10))
model.add(layers.Dense(1, activation='sigmoid'))

In [173]:
# Model compilation
model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy', metrics=['accuracy'])

In [174]:
# Callback for TensorBoard
logdir = "./logs/train"
tensorboard_callback = TensorBoard(log_dir=logdir)

In [178]:
# Directory to save gradient visualizations
save_dir = './gradient_visualizations'
os.makedirs(save_dir, exist_ok=True)

In [179]:
# Function to calculate and visualize gradients and save as PNG
def visualize_gradients_and_save(model, X, epoch):
    with tf.GradientTape() as tape:
        predictions = model(X)
        loss = tf.reduce_mean(tf.keras.losses.binary_crossentropy(y, predictions))  # Compute loss manually

    # Calculate gradients
    gradients = tape.gradient(loss, model.trainable_variables)

    # Visualize gradients
    fig, axs = plt.subplots(len(gradients), 1, figsize=(8, 6))
    for i, grad in enumerate(gradients):
        axs[i].hist(grad.numpy().flatten(), bins=30)
        axs[i].set_title(f'Layer {i+1} Gradient Distribution')
        axs[i].set_xlabel('Gradient Value')
        axs[i].set_ylabel('Count')
    
    plt.tight_layout()
    
    # Save figure as PNG
    fig.savefig(f'{save_dir}/epoch_{epoch}_gradients.png')
    plt.close(fig)  # Close the figure to free up memory


In [180]:
epochs = 1000
for epoch in range(epochs):
    model.fit(X, y, epochs=1, verbose=0, callbacks=[tensorboard_callback])
    if epoch % 100 == 0:
        print(f"Epoch {epoch}")
        visualize_gradients_and_save(model, X, epoch)


Epoch 0
Epoch 100
Epoch 200
Epoch 300
Epoch 400
Epoch 500
Epoch 600
Epoch 700
Epoch 800
Epoch 900
